In [1]:
import numpy as np, pandas as pd, bs4

In [2]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [3]:
myurl = 'https://web-as.tamu.edu/gradereports/'
registrar_page = uReq(myurl)
page_html = registrar_page.read()
registrar_page.close()
page_soup = soup(page_html, 'html.parser')
diff_distributions = page_soup.findAll('table', {})
len(diff_distributions)

7

In [4]:
index = []
for i in range(3):
    index.append([])
    for row in diff_distributions[i+1].findAll('option'):
        index[i].append(row['value'])


In [77]:

url = "https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211DN_PROF.pdf"
import tabula
df = tabula.read_pdf(url,pages = 'all')


In [78]:
cols = ['SECTION', "A", "B+",'B', "C+", "C", 'D', 'F','TOTAL A-F','GPA','I','S','U','X','TOTAL','INSTRUCTOR' ]
combined_df = pd.DataFrame(columns = cols)
for i in range(len(df)-1):
    df[i].loc[-1] = df[i].columns.values
    df[i].index = df[i].index + 1  # shifting index
    df[i].sort_index(inplace=True)
    df[i].columns = cols
    combined_df = combined_df.append(df[i], ignore_index = True)
combined_df.head()

0
1
2
3
4
5
6
7
8
9
10


,SECTION,A,B+,B,C+,C,D,F,TOTAL A-F,GPA,I,S,U,X,TOTAL,INSTRUCTOR
0,DDDS-601-300,54,0,35,0.1,14,0.2,3,106,3.292,0.3,0.4,0.5,0.6,106.1,SVOBODA K
1,NaN,50.94%,0.00%,33.02%,0.00%,13.21%,0.00%,2.83%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,COURSE TOTAL:,54,0,35,0,14,0,3,106,3.292,0,0,0,0,106,NaN
3,NaN,50.94%,0.00%,33.02%,0.00%,13.21%,0.00%,2.83%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DDDS-602-300,68,0,27,0,11,0,0,106,3.537,0,0,0,0,106,KESTERKE M


In [13]:
num_pages = pdfReader.numPages
print(num_pages)
section_rows = []
course_rows = []
for i in range(0,num_pages):
    pageObj = pdfReader.getPage(i)
    text = pageObj.extractText()
    data = text.split('\n')
    if('Undergraduate' in data):
        idx = data.index('Undergraduate')
        data = data[idx+1:]
    elif('Graduate' in data):
        idx = data.index('Graduate')
        data = data[idx+1:]
    data.pop() #removing empty record related to last space in data
    
    while(data):
        if(data[0] != 'COURSE TOTAL:'):
            row = data[0:20]
            data = data[20:]
            course_temp = row[0]
            section_rows.append(row)
        else:
            row = data[0:19]
            course_tokens = course_temp.split('-')
            row[0] = course_tokens[0] + '-' + course_tokens[1]
            data = data[19:]
            course_rows.append(row)
section_df = pd.DataFrame(section_rows, columns=['Course Name', 'A_number', 'A_percent','B_number', 'B_percent','C_number', 'C_percent','D_number', 'D_percent','F_number', 'F_percent','TotalA_F','GPA','I','S','U','Q','X','Total','Instructor'])
course_df = pd.DataFrame(course_rows, columns=['Course Name', 'A_number', 'A_percent','B_number', 'B_percent','C_number', 'C_percent','D_number', 'D_percent','F_number', 'F_percent','TotalA_F','GPA','I','S','U','Q','X','Total'])
section_df

12


,Course Name,A_number,A_percent,B_number,B_percent,C_number,C_percent,D_number,D_percent,F_number,F_percent,TotalA_F,GPA,I,S,U,Q,X,Total,Instructor
0,SECTION,B,D,F,TOTAL,A - F,GPA,I,S,U,X,TOTAL,INSTRUCTOR,------------,-----,-----,---,---,---,---
1,------,------,TEXAS A&M UNIVERSITY,GRADE DISTRIBUTION REPORT FOR FULL YR PROFESSI...,-----,-----,-----,C+,C,B+,-----,A,-----,-----,COLLEGE:,DEPARTMENT:,DENTISTRY,COLLEGE OF DENTISTRY,------------,Professional Dentistry
2,DDDS-601-300,3.292,SVOBODA K,3,106,0,0,0,0,106,2.83%,0.00%,0,14,0.00%,13.21%,0,35,0.00%,33.02%
3,0,54,50.94%,COURSE TOTAL:,54,50.94%,0,0.00%,35,33.02%,0,0.00%,14,13.21%,0,0.00%,3,106,3.292,0
4,0,0,0,106,2.83%,DDDS-602-300,3.537,KESTERKE M,0,106,0,0,0,0,106,0.00%,0.00%,0,11,0.00%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,SECTION,B,D,F,TOTAL,A - F,GPA,I,S,U,X,TOTAL,INSTRUCTOR,------------,-----,-----,---,---,---,---
166,------,------,TEXAS A&M UNIVERSITY,GRADE DISTRIBUTION REPORT FOR FULL YR PROFESSI...,-----,-----,-----,C+,C,B+,-----,A,-----,-----,COLLEGE:,DEPARTMENT:,DENTISTRY,COLLEGE OF DENTISTRY,------------,Professional Dentistry
167,DEPARTMENT TOTAL:,3880,62.90%,0,0.00%,1991,32.27%,0,0.00%,253,4.10%,0,0.00%,45,6169,3.492,193,0,0,2
168,6364,0.73%,COLLEGE TOTAL:,3880,62.90%,0,0.00%,1991,32.27%,0,0.00%,253,4.10%,0,0.00%,45,6169,3.492,193,0
